In [2]:
%matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.special

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

from tensorflow.python.client import device_lib

device_lib.list_local_devices()

Using matplotlib backend: Qt5Agg


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6042392506198790590
 xla_global_id: -1]

In [3]:
def Func(x, y):
    z = (np.sin(x) * np.exp((1 - np.cos(y)) ** 2) + np.cos(y) * np.exp((1 - np.sin(x)) ** 2) + (x - y) ** 2)
    return z

In [4]:
class NeuralNetwork:
    def __init__(self, rate, inputs, hiddens, outputs):
        #добавляем 1 вход под bias
        self.i_count=inputs+1
        self.h_count=hiddens 
        self.o_count=outputs
        #заполняем массивы весов рандомом
        self.w_ih = np.random.normal(0.0, pow(self.h_count, -0.5), (self.h_count, self.i_count))
        self.w_ho = np.random.normal(0.0, pow(self.o_count, -0.5), (self.o_count, self.h_count))
        #learning rate и сигмоид 
        self.lr = rate
        self.activation_function = lambda x: scipy.special.expit(x)
        
    def train(self, inputs_list, targets_list):
        #добавляем 1 вход под bias
        inputs_list=np.concatenate((inputs_list,[1]), axis=0)
        #вектор-столбцы входных данных и правильных ответов
        inputs = np.array(inputs_list, ndmin=2).T 
        targets = np.array(targets_list, ndmin=2).T
        #прямое распространение. сигмоид и линеар.
        hid_results = self.activation_function(np.dot(self.w_ih, inputs))
        out_results = np.dot(self.w_ho, hid_results)
        
        #ошибки вывода
        out_errors = (targets - out_results)
        #ошибки скрытого слоя
        hid_errors = np.dot(self.w_ho.T, out_errors)
        #поправки для весов скрытый-выход
        self.w_ho += self.lr * np.dot(out_errors, np.transpose(hid_results))
        #поправки для весов вход-скрытый
        self.w_ih += self.lr * np.dot((hid_errors * hid_results * (1.0 - hid_results)), np.transpose(inputs))
        
    def query(self, inputs_list):
        #добавляем 1 вход под bias
        inputs_list=np.concatenate((inputs_list,[1]), axis=0)
        #вектор-столбец входных данных
        inputs = np.array(inputs_list, ndmin=2).T 
        #прямое распространение. сигмоид и линеар.
        hid_results = self.activation_function(np.dot(self.w_ih, inputs))
        out_results = np.dot(self.w_ho, hid_results)
        return out_results
    
    def set_lr(self, rate):
        self.lr = rate


In [20]:
net = NeuralNetwork(rate=0.0001, inputs=2, hiddens=200, outputs=1)

In [21]:
xy_set = []
z_set = []
d = 10

for x in range(-5*d, 5*d):
    for y in range(-5*d, 5*d):
        xy_set += [[x/d, y/d]]
        z_set += [Func(x/d, y/d)]
        
xy_set = np.array(xy_set)
z_set = np.array(z_set)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(xy_set[:, 0], xy_set[:, 1], z_set, c='blue', marker='o')

plt.pause(5)

In [22]:
net.set_lr(0.0001)
epochs = 100
plt.ion()
fig = plt.figure()
ax = fig.gca(projection='3d')   
scat1 = ax.scatter(xy_set[:,0], xy_set[:,1], z_set, c='blue', marker='o')  
scat2 = ax.scatter(xy_set[:,0], xy_set[:,1], z_set, c='blue', marker='o')  
for e in range(epochs):
    
    for kkk in range(10):
        for x in range(10000):
            net.train(xy_set[x,:], z_set[x])
        
    MSE=0
    test_results=np.zeros(10000)
    for x in range(10000):
        result = net.query(xy_set[x,:])
        MSE+= (z_set[x] - result)**2
        test_results[x]=result
    
    scat2.remove() 
    scat2=ax.scatter(xy_set[:,0], xy_set[:,1], test_results, c='red', marker='o')     
    plt.pause(5)
    print(MSE/10000)

[[1510.22294957]]
[[1799.88787179]]
[[1621.56036043]]
[[2029.86995957]]
[[927.21426813]]
[[649.27423101]]
[[1215.80630384]]
[[540.00047175]]
[[383.22966217]]
[[330.72668131]]
[[290.50413151]]
[[325.29804721]]
[[404.68798059]]
[[375.01963416]]
[[279.79808477]]
[[197.83012261]]
[[197.14579629]]
[[185.62758937]]
[[200.90304023]]
[[199.80218547]]
[[194.45592841]]
[[196.06408633]]
[[182.36792269]]
[[190.57235988]]
[[116.09137371]]
[[129.13831706]]
[[121.24978475]]
[[137.48204391]]
[[108.30241068]]
[[102.87740089]]
[[106.58854728]]
[[96.21664292]]
[[99.84998309]]
[[94.77690422]]
[[91.12477936]]
[[120.46537771]]
[[121.47575851]]
[[142.27984634]]
[[118.46105754]]
[[94.70333735]]
[[92.64485428]]
[[93.84023662]]
[[87.60344978]]
[[85.3437972]]
[[80.77747023]]
[[67.94340577]]
[[81.87212337]]
[[78.68779346]]
[[64.00271564]]
[[55.30848824]]
[[55.86511332]]
[[59.16668934]]
[[80.20349193]]
[[80.1331973]]
[[86.19450204]]
[[84.70304865]]
[[91.49193435]]
[[97.48180765]]
[[84.0417029]]
[[83.04127809]]
[[9